In [48]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pymongo
import time

In [49]:
#Setting login values for web scrapping and chromedriver path
email = ''
password = ''
chrome_driver = ''
file = open('variables.txt', 'r') 
lines = file.readlines()
email = lines[0].split()[1].strip()
pwd = lines[1].split()[1].strip()
chrome_driver = lines[2].split()[1].strip()


Priyanka#9


In [50]:
#Launch LinkedIn using chrome driver
driver = webdriver.Chrome(chrome_driver)
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
time.sleep(5)
username_field = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[1]/input')
time.sleep(1)
username_field.send_keys(email)
password = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[2]/input')
time.sleep(1)
password.send_keys(pwd)
sign_in_button = driver.find_element_by_xpath('/html/body/div[1]/main/div[2]/form/div[4]/button')
sign_in_button.click()
time.sleep(1)